In [1]:
using CSV, DataFrames, CategoricalArrays
using Statistics
using CategoricalArrays

# Top 3

In [76]:
train_grouped = CSV.read("Training_Filtered_top3_short.csv", DataFrame)
test_grouped = CSV.read("Testing_Filtered_top3_short.csv", DataFrame)

train_grouped = train_grouped[:, 2:54]
#30756 rows

test_grouped = test_grouped[:, 2:54];
#30819 rows

In [77]:
train_X = train_grouped[:,1:52]
train_y = train_grouped[:,53];


In [78]:
test_X = test_grouped[:,1:52]
test_y = test_grouped[:,53];


## Specify treatments

In [79]:
treatment_cols =  ["med_count_1", "med_count_2", "med_count_3", "med_count_4", "med_count_5", "med_count_6"];

excluded_cols = [:med_count_1, :med_count_2, :med_count_3, 
                 :med_count_4, :med_count_5, :med_count_6, :readmitted];

In [80]:
# Select only feature columns for train and test sets
train_X = select(train_grouped, Not(excluded_cols))
test_X = select(test_grouped, Not(excluded_cols));

In [81]:
# Extract only treatment columns for train and test sets
treatments_train = select(train_grouped, treatment_cols)
treatments_test = select(test_grouped, treatment_cols);

In [82]:
train_T_categorical = string.("[", treatments_train.med_count_1, ",", treatments_train.med_count_2, ",", 
    treatments_train.med_count_3,",",treatments_train.med_count_4, ",",treatments_train.med_count_5,",",
    treatments_train.med_count_6, "]")

test_T_categorical = string.("[", treatments_test.med_count_1, ",", treatments_test.med_count_2, ",", 
    treatments_test.med_count_3,",",treatments_test.med_count_4, ",",treatments_test.med_count_5,",",
    treatments_test.med_count_6, "]");


In [83]:
# Extract the outcome variable
train_y = train_grouped[:, :readmitted]
test_y = test_grouped[:, :readmitted];

## Optimal Policy Trees

In [84]:
seed=42

categorical_reward_lnr = IAI.CategoricalClassificationRewardEstimator(
    propensity_estimator=IAI.RandomForestClassifier(random_seed=seed),
    outcome_estimator=IAI.RandomForestClassifier(random_seed=seed),
    reward_estimator=:doubly_robust,
    random_seed=seed,
)

Unfitted CategoricalClassificationRewardEstimator:
  propensity_estimator: Unfitted RandomForestClassifier:
    random_seed: 42
  outcome_estimator:    Unfitted RandomForestClassifier:
    random_seed: 42
  reward_estimator:     doubly_robust
  random_seed:          42

### Training Set

In [85]:
train_predictions, train_reward_score = IAI.fit_predict!(
    categorical_reward_lnr, train_X, train_T_categorical, train_y)

train_rewards = train_predictions[:reward]
train_reward_score[:outcome]


Dict{String, Float64} with 3 entries:
  "[0,0,0,0,0,0]" => 0.0885725
  "[0,1,0,0,0,0]" => 0.0558424
  "[0,0,0,0,0,1]" => 0.077174

In [86]:
train_reward_score[:propensity]
#If outcomes are good, and propsity is bad--> can go ahead 

0.5089521853665767

### Testing Set

In [87]:
test_predictions, test_reward_score = IAI.fit_predict!(
    categorical_reward_lnr, test_X, test_T_categorical, test_y)

test_rewards = test_predictions[:reward]
test_reward_score[:outcome]

Dict{String, Float64} with 3 entries:
  "[0,0,0,0,0,0]" => 0.0851641
  "[0,1,0,0,0,0]" => 0.0672116
  "[0,0,0,0,0,1]" => 0.0811282

In [88]:
test_reward_score[:propensity]


0.5013239080233148